#1.   Download the dataset from your dataset URL.

In [45]:
# Install the Kaggle library
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
# Make a directory named “.kaggle”
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [47]:
# Copy the “kaggle.json” into this new directory
!cp kaggle.json ~/.kaggle/

In [48]:
# Allocate the required permission for this file.
!chmod 600 ~/.kaggle/kaggle.json

In [50]:
# Download the required file
! kaggle competitions download -c sentiment-analysis-of-covid-19-related-tweets

! kaggle datasets download gpreda/covid19-tweets

sentiment-analysis-of-covid-19-related-tweets.zip: Skipping, found more recently modified local copy (use --force to force download)


In [56]:
# Unzip all the downloaded files
!unzip sentiment-analysis-of-covid-19-related-tweets.zip
!unzip covid19-tweets.zip

Archive:  sentiment-analysis-of-covid-19-related-tweets.zip
replace training.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  covid19-tweets.zip
replace covid19_tweets.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


#2.   Read external sources for the dataset to understand the schema and data.

In [63]:
!ls -l

total 169148
-rw-r--r-- 1 root root 68713814 Aug 30  2020 covid19_tweets.csv
-rw-r--r-- 1 root root 74043392 Mar 23 04:39 covid19_tweets.db
-rw-r--r-- 1 root root 29305228 Mar 23 04:55 covid19-tweets.zip
-rw-r--r-- 1 root root       66 Mar 23 04:52 kaggle.json
drwxr-xr-x 1 root root     4096 Mar 21 22:54 sample_data
-rw-r--r-- 1 root root   339144 Mar 23 04:53 sentiment-analysis-of-covid-19-related-tweets.zip
-rw-r--r-- 1 root root   530787 May 10  2021 training.csv
-rw-r--r-- 1 root root   260187 May 10  2021 validation.csv


In [59]:
# Read covid19_tweets
!head -n 10 covid19_tweets.csv | column -t -s , 

user_name                                                                    user_location           user_description                                                             user_created                                                                                                                   user_followers       user_friends                                                                                                                                user_favourites      user_verified                                                                                                                            date                 text                                                                                                                                         hashtags                                                                                                                                      source                                                                  

In [62]:
# Read training
!head -n 10 training.csv | column -t -s , 

ID  Tweet                                                                                                               Labels
1   NO JOKE I WILL HOP ON A PLANE RN! (Well after COVID19 lol)                                                          0 10
2   BanMediaHouse whose is responsible for spreading Fake and communal Stories in this pandemic Corona situation.       6
3   Im waiting for someone to say to me that all this corona thing is just an April fools joke                          3 4
4   He is a liar. Proven day night. Time again. Lies when the truth will do. COVID19                                    6
5   "NEW: U.S. CoronaVirus death toll reaches 4                                                                         000 after nearly 900 new deaths were reported today (BNO News) Covid19 CoronaVirusOutbreak"           8
6   "Coronavirus impact Govt extends I-T deadlines related to Sections 80C                                               80D"                               

In [61]:
# Read validation
!head -n 10 validation.csv | column -t -s , 

ID    Tweet
5001  "Forgot to a math test and I was failing but my teacher opening it back up                                maybe corona aint so bad"
5002  Corona effected came from Delhi in Bihar lakhisarai district at Mahisona village his name is pappu Khan
5003  Make CORONA END NA
5004  Imagine if the coronavirus pandemic was a big April fools joke the government pulled on us .....
5005  Howdy Ana. Where can I get married during the coronavirus? Any update from the Gov?
5006  Coronavirus live blog: Allergist and immunologist Dr. Purvi Parikh answers your questions
5007  We won't b talking about corona but about hunger killing pple
5008  Fuck Corona man... I couldve gone on a vacation next week to the states if this shit doesnt exist
5009  "Female-founded companies                                                                                 as a result                 have less cash on hand to carry them through any kind of unexpected event   like COVID19."


#3.   Create a database to store the data, using either SQL or NoSQL.

In [1]:
!sudo apt-get install sqlite3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
sqlite3 is already the newest version (3.31.1-4ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [2]:
!touch covid19_tweets.db

In [3]:
%load_ext sql

In [4]:
%sql sqlite:///covid19_tweets.db

'Connected: @covid19_tweets.db'

In [21]:
# %%sql

# DROP TABLE training_tweets;

 * sqlite:///covid19_tweets.db
Done.


[]

#4.   Insert the data into the database, using either SQL or NoSQL.
#5.   Connect to the database using Python and retrieve the data from the tables, using either SQL or NoSQL.
#6.   Save the data into a pandas dataframe or other format.

In [7]:
# Create a table to store the CSV data

%%sql
CREATE TABLE tweets (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_name VARCHAR(255),
    user_location VARCHAR(255),
    user_description TEXT,
    user_created DATETIME,
    user_followers INT,
    user_friends INT,
    user_favourites INT,
    user_verified BOOLEAN,
    date DATETIME,
    text TEXT,
    hashtags VARCHAR(255),
    source VARCHAR(255),
    is_retweet BOOLEAN);

 * sqlite:///covid19_tweets.db
Done.


[]

In [10]:
!sqlite3 -separator ',' covid19_tweets.db ".import covid19_tweets.csv tweets"


In [6]:
%%sql
.separator ','

.import covid19_tweets.csv tweets

 * sqlite:///covid19_tweets.db
(sqlite3.OperationalError) near ".": syntax error
[SQL: .separator ','

.import covid19_tweets.csv tweets]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [35]:
import sqlite3
import csv

# Connect to the database
conn = sqlite3.connect('covid19_tweets.db')

# Create a cursor object
cursor = conn.cursor()

In [26]:
# Create a table to store the CSV data
create_table_query = '''
    CREATE TABLE training_tweets (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Tweet TEXT,
    Labels VARCHAR(255)
);
'''
cursor.execute(create_table_query)

In [27]:

# Execute the SQL command to insert data from CSV file
with open('training.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # skip header row
    for row in reader:
        cursor.execute('INSERT INTO training_tweets (id, Tweet, Labels) VALUES (?, ?, ?)', row)

# Commit the changes
conn.commit()

In [36]:
# Filtering the Optimistic (1) labeling training data
cursor = conn.execute("SELECT * from training_tweets WHERE Labels=1")
for row in cursor:
    print(row)
rows = cursor.fetchmany(10)
# Close the connection to the database


(343, 'Shout out to !!! I truly respect the way youre leading NY through this crisis!!! COVID19', '1')
(453, "I'll say it again, good work ! This is LEADERSHIP in the MEDIA!! MembersOfThePressWTF COVID19 VoteBlue2020", '1')
(581, 'Thank you so much for this UAE! indeed an answered prayer coronavirus Covid_19', '1')
(582, 'My mentor is out there fighting coronavirus in Detroit hospitals and I dont think Ive ever been prouder', '1')
(711, 'Three teams are fighting Covid19 day night: 1: Doctors, Nurses, Health Officials. 2: Modis PR team. 3: IT cell', '1')
(715, 'thanks to and for a brilliant zoom on covid19 emergency payment and tax and everything x', '1')
(1094, 'Thank you and G42! An impressive feat! Coronavirus diagnostics are ramped up in the UAE', '1')
(1281, 'When your congressman deploys with the to help fight the coronavirus . Thats leading by example.', '1')
(1356, 'Kylie Jenner to produce and donate hand sanitizer to hospitals battling covid19. Now thats an influencer.', '1')
(

In [38]:
# Retrieve data from the table
cursor = conn.execute("SELECT * from tweets")
rows = cursor.fetchmany(10)
rows

[('ᏉᎥ☻լꂅϮ',
  'astroworld',
  'wednesday addams as a disney princess keepin it [̲̅$̲̅(̲̅ιοο̲̅)̲̅$̲̅] 👸🏻💚♊️🧀🌵🌃🌙',
  '2017-05-26 05:46:42',
  '624',
  '950',
  '18775',
  'False',
  '2020-07-25 12:27:21',
  'If I smelled the scent of hand sanitizers today on someone in the past, I would think they were so intoxicated that… https://t.co/QZvYbrOgb0',
  '',
  'Twitter for iPhone',
  'False'),
 ('Tom Basile 🇺🇸',
  'New York, NY',
  'Husband, Father, Columnist & Commentator. Author of Tough Sell: Fighting the Media War in Iraq. Bush Admin Alum. Newsmax Contributor. Fmr Exec Dir NYSGOP',
  '2009-04-16 20:06:23',
  '2253',
  '1677',
  '24',
  'True',
  '2020-07-25 12:27:17',
  "Hey @Yankees @YankeesPR and @MLB - wouldn't it have made more sense to have the players pay their respects to the A… https://t.co/1QvW0zgyPu",
  '',
  'Twitter for Android',
  'False'),
 ('Time4fisticuffs',
  'Pewee Valley, KY',
  '#Christian #Catholic #Conservative #Reagan #Republican #Capitalist; Sports lover - #BBN #C

#7.   Perform exploratory data analysis (EDA) on the data to understand the distribution, trends, and relationships among variables.
#8.   Visualize the data using matplotlib, seaborn, or other visualization libraries.


In [32]:

conn.commit()
conn.close()

In [62]:


import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import string
from wordcloud import WordCloud, STOPWORDS
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

#9.   Apply data analysis techniques like sentiment analysis, topic modeling, or network analysis to draw insights from the data and write your evaluation about the results.
#10.   Write the results back to the database.

In [64]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [52]:
class_map = {
    0: "optimistic",
    1: "thankful",
    2: "empathetic",
    3: "pessimistic",
    4: "anxious",
    5: "sad",
    6: "annoyed",
    7: "denial",
    8: "surprise",
    9: "official_report",
    10: "joking"
}

inv_class_map = {
    "optimistic": 0,
    "thankful": 1,
    "empathetic": 2,
    "pessimistic": 3,
    "anxious": 4,
    "sad": 5,
    "annoyed": 6,
    "denial": 7,
    "surprise": 8,
    "official_report": 9,
    "joking": 10
}

In [59]:
RANDOM_SEED = 42
CLASSIFIER_THRESHOLD = 0.4

np.random.seed(RANDOM_SEED)

In [56]:
records = []
with open('training.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            current_classes = []
            classes = list(map(int, row[2].split(" ")))
            for i in range(0, 11):
                if i in classes:
                    current_classes.append(class_map[i])

            records.append((row[0], row[1], current_classes))
            line_count += 1

    print(f'Processed {line_count} lines.')

training_df = pd.DataFrame.from_records(records, columns=["id", "text", "labels"])

df = pd.DataFrame.from_records(records, columns=["id", "text", "labels"])

#trainDataset, testDataset = spark.read.parquet("training_sparknlp.parquet").randomSplit([0.9, 0.1], seed=RANDOM_SEED)
trainDataset, testDataset = spark.createDataFrame(df).randomSplit([0.9, 0.1], seed=RANDOM_SEED)

#df.to_parquet("training_sparknlp.parquet", compression="gzip")
df.info()

Column names are ID, Tweet, Labels
Processed 5001 lines.


In [58]:
training_df.head(2)

,id,text,labels
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,"[optimistic, joking]"
1,2,BanMediaHouse whose is responsible for spreadi...,[annoyed]


In [43]:
tweet_df_filepath = "covid19_tweets.csv"

training_df = pd.read_csv("training.csv")
tweet_df = pd.read_csv(tweet_df_filepath)

In [44]:
tweet_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False


In [27]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179108 entries, 0 to 179107
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_name         179108 non-null  object
 1   user_location     142337 non-null  object
 2   user_description  168822 non-null  object
 3   user_created      179108 non-null  object
 4   user_followers    179108 non-null  int64 
 5   user_friends      179108 non-null  int64 
 6   user_favourites   179108 non-null  int64 
 7   user_verified     179108 non-null  bool  
 8   date              179108 non-null  object
 9   text              179108 non-null  object
 10  hashtags          127774 non-null  object
 11  source            179031 non-null  object
 12  is_retweet        179108 non-null  bool  
dtypes: bool(2), int64(3), object(8)
memory usage: 15.4+ MB


In [28]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [89]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [45]:
tweet_df.loc[tweet_df.hashtags.isna()]

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
7,Derbyshire LPC,NaN,NaN,2012-02-03 18:08:10,608,355,95,False,2020-07-25 12:27:00,You now have to wear face coverings when out s...,NaN,TweetDeck,False
10,Voice Of CBSE Students,NaN,NaN,2020-07-14 17:50:30,8,10,7,False,2020-07-25 12:26:53,49K+ Covid19 cases still no response from \n@c...,NaN,Twitter Web App,False
14,DailyaddaaNews,New Delhi,Breaking news alerts from India.,2016-10-22 09:18:42,546,29,88,False,2020-07-25 12:26:46,Rajasthan Government today started a Plasma Ba...,NaN,Twitter Web App,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179085,3D Innovations,"Honolulu, Hawaii","Providing Engineering/3D Design, 3D Printing, ...",2011-07-06 19:50:05,5776,3762,13554,False,2020-08-29 19:45:01,How toilet paper and avocados help explain the...,NaN,Sprout Social,False
179090,Randy Richardson,"Des Moines, IA",Organizer with Iowans for Public Ed. Former (a...,2009-04-13 20:33:21,2873,4846,16713,False,2020-08-29 19:44:53,Johnston school district currently has 4 stude...,NaN,Twitter for Android,False
179096,laura l. sweet,"Los Angeles, CA","Creative Director/AD/CW, Advertising & Brandin...",2008-03-18 16:45:55,2146,1077,5306,False,2020-08-29 19:44:40,We were really bummed we couldn’t cop one of t...,NaN,Instagram,False
179105,BEEHEMOTH ⏳,🇨🇦 Canada,⚒️ The Architects of Free Trade ⚒️ Really Did ...,2016-07-13 17:21:59,1623,2160,98000,False,2020-08-29 19:44:15,@CTVNews A powerful painting by Juan Lucena. I...,NaN,Twitter Web App,False


In [31]:
tweet_df.hashtags.isnull().sum()


51334

In [32]:
# tweet_df.loc[tweet_df.hashtags.isna()]['text'].apply(lambda x : extract_hash_tags(x))

0          
1          
7          
10         
14         
         ..
179085     
179090     
179096     
179105     
179107     
Name: text, Length: 51334, dtype: object

In [50]:
def extract_hash_tags(s):
    hashes = re.findall(r"#(\w+)", s)
    if hashes:
      return " ".join(hashes)
    else:
      return None
tweet_df['new_hashtags'] = tweet_df['text'].apply(lambda x : extract_hash_tags(x))

In [47]:
tweet_df.hashtags.isnull().sum()

51334

In [51]:
tweet_df

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,new_hashtags
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False,None
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False,None
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False,COVID19
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False,COVID19
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False,CoronaVirusUpdates COVID19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179103,AJIMATI AbdulRahman O.,"Ilorin, Nigeria",Animal Scientist|| Muslim|| Real Madrid/Chelsea,2013-12-30 18:59:19,412,1609,1062,False,2020-08-29 19:44:21,Thanks @IamOhmai for nominating me for the @WH...,['WearAMask'],Twitter for Android,False,WearAMask
179104,Jason,Ontario,When your cat has more baking soda than Ninja ...,2011-12-21 04:41:30,150,182,7295,False,2020-08-29 19:44:16,2020! The year of insanity! Lol! #COVID19 http...,['COVID19'],Twitter for Android,False,COVID19
179105,BEEHEMOTH ⏳,🇨🇦 Canada,⚒️ The Architects of Free Trade ⚒️ Really Did ...,2016-07-13 17:21:59,1623,2160,98000,False,2020-08-29 19:44:15,@CTVNews A powerful painting by Juan Lucena. I...,NaN,Twitter Web App,False,None
179106,Gary DelPonte,New York City,"Global UX UI Visual Designer. StoryTeller, Mus...",2009-10-27 17:43:13,1338,1111,0,False,2020-08-29 19:44:14,"More than 1,200 students test positive for #CO...",['COVID19'],Twitter for iPhone,False,COVID19


In [35]:
tweet_df['text'] = tweet_df['text'].apply(lambda x: clean_text(x))

In [90]:
tweet_df['text'] = tweet_df['text'].apply(lambda x: remove_emoji(x))

In [65]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

In [86]:
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)
    

In [96]:
import re


def remove_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)



In [91]:
tweet_df.text = tweet_df.text.apply(lambda x: preprocess(x))
tweet_df['text'] = tweet_df['text'].apply(lambda x: clean_text(x))
tweet_df['text'] = tweet_df['text'].apply(lambda x: remove_emoji(x))


In [102]:
tweet_df['text'] = tweet_df['text'].apply(lambda x: remove_emoji(x))


In [105]:
tweet_df

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,new_hashtags,Labels
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,smelled scent hand sanitizers today someone pa...,NaN,Twitter for iPhone,False,None,10
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,hey yankees yankeespr mlb made sense players p...,NaN,Twitter for Android,False,None,6
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,wdunlap realdonaldtrump trump never claimed ho...,['COVID19'],Twitter for Android,False,COVID19,6
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,one gift give appreciation simple things alway...,['COVID19'],Twitter for iPhone,False,COVID19,6
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,july media bulletin novel coronavirusupdates d...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False,CoronaVirusUpdates COVID19,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179103,AJIMATI AbdulRahman O.,"Ilorin, Nigeria",Animal Scientist|| Muslim|| Real Madrid/Chelsea,2013-12-30 18:59:19,412,1609,1062,False,2020-08-29 19:44:21,thanks iamohmai nominating wearamask challenge...,['WearAMask'],Twitter for Android,False,WearAMask,6 10
179104,Jason,Ontario,When your cat has more baking soda than Ninja ...,2011-12-21 04:41:30,150,182,7295,False,2020-08-29 19:44:16,year insanity lol,['COVID19'],Twitter for Android,False,COVID19,10
179105,BEEHEMOTH ⏳,🇨🇦 Canada,⚒️ The Architects of Free Trade ⚒️ Really Did ...,2016-07-13 17:21:59,1623,2160,98000,False,2020-08-29 19:44:15,powerful painting juan lucena tribute grandpar...,NaN,Twitter Web App,False,None,6
179106,Gary DelPonte,New York City,"Global UX UI Visual Designer. StoryTeller, Mus...",2009-10-27 17:43:13,1338,1111,0,False,2020-08-29 19:44:14,students test positive major university abc news,['COVID19'],Twitter for iPhone,False,COVID19,8


In [106]:
df_train

,ID,Tweet,Labels
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,0 10
1,2,BanMediaHouse whose is responsible for spreadi...,6
2,3,Im waiting for someone to say to me that all t...,3 4
3,4,He is a liar. Proven day night. Time again. Li...,6
4,5,"NEW: U.S. CoronaVirus death toll reaches 4,000...",8
...,...,...,...
4995,4996,Life Insurance? I wonder if policies are payin...,4 5 7
4996,4997,My cousin passed away from the corona virus to...,4 5
4997,4998,I guess Mother Nature really hates us. Yellows...,3 5 9 10
4998,4999,So question for the day that isnt related to c...,9 10


KeyError: ignored

In [78]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
df_train = pd.read_csv('training.csv')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_train['Tweet'], df_train['Labels'], test_size=0.2, random_state=42)

# Convert the text data to numerical features using CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train a Naive Bayes classifier on the training data
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vec, y_train)

# Make predictions on the testing data
y_pred = nb_classifier.predict(X_test_vec)

# Evaluate the accuracy and other metrics of the classifier
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:\n', classification_report(y_test, y_pred))


Accuracy: 0.156
Classification Report:
               precision    recall  f1-score   support

           0       0.35      0.18      0.24        33
         0 1       0.00      0.00      0.00        14
       0 1 2       0.00      0.00      0.00         3
       0 1 4       0.00      0.00      0.00         1
       0 1 5       0.00      0.00      0.00         1
    0 1 5 10       0.00      0.00      0.00         1
       0 1 6       0.00      0.00      0.00         1
     0 1 6 7       0.00      0.00      0.00         1
    0 1 7 10       0.00      0.00      0.00         1
       0 1 8       0.00      0.00      0.00         1
       0 1 9       0.00      0.00      0.00         5
        0 10       0.10      0.22      0.14        49
         0 2       0.00      0.00      0.00         9
      0 2 10       0.00      0.00      0.00         3
       0 2 5       0.00      0.00      0.00         2
     0 2 7 9       0.00      0.00      0.00         1
         0 3       0.00      0.00      0.

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [80]:
train_data

,ID,Tweet,Labels
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,0 10
1,2,BanMediaHouse whose is responsible for spreadi...,6
2,3,Im waiting for someone to say to me that all t...,3 4
3,4,He is a liar. Proven day night. Time again. Li...,6
4,5,"NEW: U.S. CoronaVirus death toll reaches 4,000...",8
...,...,...,...
4995,4996,Life Insurance? I wonder if policies are payin...,4 5 7
4996,4997,My cousin passed away from the corona virus to...,4 5
4997,4998,I guess Mother Nature really hates us. Yellows...,3 5 9 10
4998,4999,So question for the day that isnt related to c...,9 10


In [107]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Load the data
df_train = pd.read_csv('training.csv')
df_test = pd.read_csv('validation.csv')

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df_train['Tweet'], df_train['Labels'], test_size=0.2, random_state=42)

# Transform the text data into numerical feature vectors
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)
X_test_vec = vectorizer.transform(df_test['Tweet'])

# Train a Naive Bayes classifier on the training set and evaluate on the validation set
clf = MultinomialNB()
clf.fit(X_train_vec, y_train)
y_pred = clf.predict(X_val_vec)
print(classification_report(y_val, y_pred))

# Make predictions on the test set and save the results to a CSV file
df_test['Labels'] = clf.predict(X_test_vec)
df_test.to_csv('test_results.csv', index=False)


              precision    recall  f1-score   support

           0       0.33      0.09      0.14        33
         0 1       0.00      0.00      0.00        14
       0 1 2       0.00      0.00      0.00         3
       0 1 4       0.00      0.00      0.00         1
       0 1 5       0.00      0.00      0.00         1
    0 1 5 10       0.00      0.00      0.00         1
       0 1 6       0.00      0.00      0.00         1
     0 1 6 7       0.00      0.00      0.00         1
    0 1 7 10       0.00      0.00      0.00         1
       0 1 8       0.00      0.00      0.00         1
       0 1 9       0.00      0.00      0.00         5
        0 10       0.06      0.12      0.08        49
         0 2       0.00      0.00      0.00         9
      0 2 10       0.00      0.00      0.00         3
       0 2 5       0.00      0.00      0.00         2
     0 2 7 9       0.00      0.00      0.00         1
         0 3       0.00      0.00      0.00         4
      0 3 10       0.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [110]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Load the data
df_train = pd.read_csv('training.csv')
df_test = tweet_df

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df_train['Tweet'], df_train['Labels'], test_size=0.2, random_state=42)

# Transform the text data into numerical feature vectors
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)
X_test_vec = vectorizer.transform(df_test['text'])

# Train a Naive Bayes classifier on the training set and evaluate on the validation set
clf = MultinomialNB()
clf.fit(X_train_vec, y_train)
y_pred = clf.predict(X_val_vec)
print(classification_report(y_val, y_pred, labels=np.unique(y_pred)))

# Make predictions on the test set and save the results to a CSV file
df_test['Labels'] = clf.predict(X_test_vec)
df_test.to_csv('test_results.csv', index=False)


              precision    recall  f1-score   support

           0       0.33      0.09      0.14        33
         0 1       0.00      0.00      0.00        14
        0 10       0.06      0.12      0.08        49
          10       0.14      0.81      0.24        94
         4 5       1.00      0.07      0.13        14
         5 6       0.00      0.00      0.00        31
           6       0.10      0.47      0.17        49
        6 10       0.06      0.04      0.05        69
           8       0.57      0.93      0.70        42

   micro avg       0.15      0.38      0.22       395
   macro avg       0.25      0.28      0.17       395
weighted avg       0.19      0.38      0.19       395



In [113]:
df_test.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,new_hashtags,Labels
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,smelled scent hand sanitizers today someone pa...,NaN,Twitter for iPhone,False,None,10
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,hey yankees yankeespr mlb made sense players p...,NaN,Twitter for Android,False,None,10
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,wdunlap realdonaldtrump trump never claimed ho...,['COVID19'],Twitter for Android,False,COVID19,6
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,one gift give appreciation simple things alway...,['COVID19'],Twitter for iPhone,False,COVID19,6
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,july media bulletin novel coronavirusupdates d...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False,CoronaVirusUpdates COVID19,6 7


In [108]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Load the data
df_train = pd.read_csv('training.csv')
df_test = tweet_df

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df_train['Tweet'], df_train['Labels'], test_size=0.2, random_state=42)

# Transform the text data into numerical feature vectors
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)
X_test_vec = vectorizer.transform(df_test['text'])

# Train a Naive Bayes classifier on the training set and evaluate on the validation set
clf = MultinomialNB()
clf.fit(X_train_vec, y_train)
y_pred = clf.predict(X_val_vec)
print(classification_report(y_val, y_pred))

# Make predictions on the test set and save the results to a CSV file
df_test['Labels'] = clf.predict(X_test_vec)
df_test.to_csv('test_results.csv', index=False)


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.33      0.09      0.14        33
         0 1       0.00      0.00      0.00        14
       0 1 2       0.00      0.00      0.00         3
       0 1 4       0.00      0.00      0.00         1
       0 1 5       0.00      0.00      0.00         1
    0 1 5 10       0.00      0.00      0.00         1
       0 1 6       0.00      0.00      0.00         1
     0 1 6 7       0.00      0.00      0.00         1
    0 1 7 10       0.00      0.00      0.00         1
       0 1 8       0.00      0.00      0.00         1
       0 1 9       0.00      0.00      0.00         5
        0 10       0.06      0.12      0.08        49
         0 2       0.00      0.00      0.00         9
      0 2 10       0.00      0.00      0.00         3
       0 2 5       0.00      0.00      0.00         2
     0 2 7 9       0.00      0.00      0.00         1
         0 3       0.00      0.00      0.00         4
      0 3 10       0.00    

In [109]:
df_test

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,new_hashtags,Labels
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,smelled scent hand sanitizers today someone pa...,NaN,Twitter for iPhone,False,None,10
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,hey yankees yankeespr mlb made sense players p...,NaN,Twitter for Android,False,None,10
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,wdunlap realdonaldtrump trump never claimed ho...,['COVID19'],Twitter for Android,False,COVID19,6
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,one gift give appreciation simple things alway...,['COVID19'],Twitter for iPhone,False,COVID19,6
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,july media bulletin novel coronavirusupdates d...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False,CoronaVirusUpdates COVID19,6 7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179103,AJIMATI AbdulRahman O.,"Ilorin, Nigeria",Animal Scientist|| Muslim|| Real Madrid/Chelsea,2013-12-30 18:59:19,412,1609,1062,False,2020-08-29 19:44:21,thanks iamohmai nominating wearamask challenge...,['WearAMask'],Twitter for Android,False,WearAMask,0 10
179104,Jason,Ontario,When your cat has more baking soda than Ninja ...,2011-12-21 04:41:30,150,182,7295,False,2020-08-29 19:44:16,year insanity lol,['COVID19'],Twitter for Android,False,COVID19,10
179105,BEEHEMOTH ⏳,🇨🇦 Canada,⚒️ The Architects of Free Trade ⚒️ Really Did ...,2016-07-13 17:21:59,1623,2160,98000,False,2020-08-29 19:44:15,powerful painting juan lucena tribute grandpar...,NaN,Twitter Web App,False,None,6
179106,Gary DelPonte,New York City,"Global UX UI Visual Designer. StoryTeller, Mus...",2009-10-27 17:43:13,1338,1111,0,False,2020-08-29 19:44:14,students test positive major university abc news,['COVID19'],Twitter for iPhone,False,COVID19,8


In [81]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# Load the training data
train_data = pd.read_csv('training.csv')

# Split the data into features and labels
train_texts = train_data['Tweet'].values
train_labels = train_data['Labels'].values

# Tokenize the training texts
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

# Convert the texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)

# Pad the sequences to have the same length
train_padded = pad_sequences(train_sequences, maxlen=100, padding='post', truncating='post')

# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=100, input_length=100))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(11, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_padded, train_labels, epochs=10, batch_size=32, validation_split=0.2)

# Load the validation data
val_data = pd.read_csv('validation.csv')

# Split the data into features and labels
val_texts = val_data['Tweet text'].values

# Tokenize the validation texts
val_sequences = tokenizer.texts_to_sequences(val_texts)

# Pad the sequences to have the same length
val_padded = pad_sequences(val_sequences, maxlen=100, padding='post', truncating='post')

# Make predictions on the validation data
predictions = model.predict(val_padded)

# Print the predicted labels for the first 10 tweets
print(predictions[:10])


Epoch 1/10


UnimplementedError: ignored